# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

BFor this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [1]:
## Connect to DB server on AWS
import mysql.connector 
import datetime
import pandas as pd
from mysql.connector import errorcode
import config

cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    password = config.password
)
cursor = cnx.cursor()


In [ ]:
## Create new DB 


q1 = 'CREATE DATABASE project1'

cursor.execute(q1)


In [2]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'project1'
)
cursor = cnx.cursor()

In [4]:
# Create a table for the Businesses
cursor.execute("""CREATE TABLE businesses (
business_id varchar(200),
name varchar(50),
rating float,
price float
);

""")

In [ ]:
# Create a table for the reviews

q3 = """CREATE TABLE reviews (
business_id varchar(200),
name varchar(50),
review varchar(500),
review_count int
);

"""

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [54]:
# Write a function to make a call to the yelp API
import requests
import json


url = 'https://api.yelp.com/v3/businesses/search'

client_id = config.client_id
api_key = config.api_key

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

# what type of business do you want to search
term = 'diners'

#where do you want to perform this search
location = 'Bedford-Stuyvesant'

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
                'offset' : 0
:            }




response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])


<Response [200]>
<class 'str'>
{"businesses": [{"id": "l2u-IHL-rNz0uvn6-UC-Rw", "alias": "diner-brooklyn", "name": "Diner", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/RPVDxSRkzIWB1iw1Q-7FRQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/diner-brooklyn?adjust_creative=wmkiJaufttIjsNPsauGcnA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=wmkiJaufttIjsNPsauGcnA", "review_count": 745, "categories": [{"alias": "diners", "title": "Diners"}, {"alias": "newamerican", "title": "American (New)"}, {"alias": "breakfast_brunch", "title": "Breakfast & Brunch"}], "rating": 4.0, "coordinates": {"latitude": 40.71069, "longitude": -73.96559}, "transactions": [], "price": "$$", "location": {"address1": "85 Broadway", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11249", "country": "US", "state": "NY", "display_address": ["85 Broadway", "Brooklyn, NY 11249"]}, "phone": "+17184863077", "display_phone": "(718) 486-3077", "distance": 3476.922

In [55]:
data = json.loads(response.text)

In [56]:
for key in data.keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    

businesses
<class 'list'>
total
<class 'int'>
region
<class 'dict'>


In [57]:
data['total']

3100

In [58]:
data['businesses'][0]

{'id': 'l2u-IHL-rNz0uvn6-UC-Rw',
 'alias': 'diner-brooklyn',
 'name': 'Diner',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/RPVDxSRkzIWB1iw1Q-7FRQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/diner-brooklyn?adjust_creative=wmkiJaufttIjsNPsauGcnA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=wmkiJaufttIjsNPsauGcnA',
 'review_count': 745,
 'categories': [{'alias': 'diners', 'title': 'Diners'},
  {'alias': 'newamerican', 'title': 'American (New)'},
  {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.71069, 'longitude': -73.96559},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '85 Broadway',
  'address2': '',
  'address3': '',
  'city': 'Brooklyn',
  'zip_code': '11249',
  'country': 'US',
  'state': 'NY',
  'display_address': ['85 Broadway', 'Brooklyn, NY 11249']},
 'phone': '+17184863077',
 'display_phone': '(718) 486-3077',
 'distance': 3476.922515270075}

In [59]:
type(data['businesses'])

list

In [60]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

business = data['businesses'][2]

biz_tuple = (business['id'], 
             business['name'], 
             business['review_count'],
             business['price'],
             business['rating'])
biz_tuple

('Vx9jsY90nQ9PCKBa1aCt9w', "Tom's", 1287, '$$', 4.0)

In [47]:
bla = business.get('bla', 1)

In [48]:
business['bla']

AttributeError: 'Response' object has no attribute 'hasMore'

In [61]:
# Write a function to take your parsed data and insert it into the DB
def parse_data(datayelp):
    # create a container to hold our parsed data
    listdata = []
    
    
    # loop through our business and parse each individual business
    for business in datayelp:
        biz_tuple = (business['id'], 
             business['name'], 
             business['review_count'],
             business.get('price',None),
             business['rating'])
        listdata.append(biz_tuple)
    return listdata
    # return the container with all of the parsed results
    

In [63]:
parsed_data = parse_data(data['businesses'])
counter = len(parsed_data) 
url_params['offset'] =counter

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50